### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.production-product"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
production_product = df_processados['production_product']

print(f"Colunas: {production_product.shape[1]}\nLinhas: {production_product.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'productid'
duplicadas = production_product[
    production_product.duplicated(subset=['productid'], keep=False)
]

# Verificar se existem duplicadas
if not duplicadas.empty:
    # Ordenar duplicadas por 'productid' e 'modifieddate'
    duplicadas_ordenadas = duplicadas.sort_values(by=['productid', 'modifieddate'])

    # Exibir duplicadas ordenadas
    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

production_product = production_product.drop_duplicates(subset=['productid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(production_product)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = production_product.copy()


#doc: bkp = backup (cópia)

In [ ]:
# Ordenar e exibir o df por 'productid'
production_product = production_product.sort_values(by=['productid'])

print(production_product)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in production_product.columns:   
    missing_rows = production_product[production_product[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")



#doc: apesar de apresentar várias colunas com muitas linhas ausentes, não há a necessidade de tratá-las, pois não interferem negativamente à análise        

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = production_product.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)



In [ ]:
# Verificar se a coluna é toda nula
is_all_null = production_product['discontinueddate'].isnull().all()
if is_all_null:
    print("A coluna 'discontinueddate' é completamente NULL.")
else:
    print("A coluna 'discontinueddate' possui valores não nulos.")


In [ ]:
# Remover a coluna 'discontinueddate' por ser completamente NULL
production_product.drop(columns=['discontinueddate'], inplace=True)


In [ ]:
# Listar colunas binárias esperadas e verificar valores únicos e suas distribuições
coluna_binaria = ['makeflag', 'finishedgoodsflag']

for col in coluna_binaria:
    unique_values = production_product[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")
    print(f"Distribuição de '{col}':")
    print(production_product[col].value_counts())
    print()


#doc: garantir que colunas binárias contenham apenas valores esperados, no caso True ou False e identificar anomalias verificando os valores únicos em cada coluna; valores fora do padrão binário, facilitando a validação e correção


In [ ]:
#verificar informações do df
production_product.info()

In [ ]:
# Avaliando as variáveis qualitativas

coluna_qualitativa = ["color", "size", "sizeunitmeasurecode", "weightunitmeasurecode", "productline", "class", "style"]

for col in coluna_qualitativa:
    counts = production_product[col].value_counts().nlargest(10)
    percentages = (counts / production_product.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    print(f"Análise da coluna '{col}':")
    print(summary)
    print("\n")


#doc: qdte. = quantidade
#doc*: variáveis qualitativas são um tipo de variável estatística que representam características ou atributos dos dados, sem serem medidas numericamente no nosso caso, city, por exemplo

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar o estilo do gráfico para fundo escuro
sns.set(style='darkgrid', rc={"axes.facecolor": "black", "figure.facecolor": "black"})

coluna_qualitativa = ["color", "size", "weightunitmeasurecode", "productline", "class", "style"]

for col in coluna_qualitativa:
    counts = production_product[col].value_counts().nlargest(10)  # Top 10 categorias
    percentages = (counts / production_product.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    
    # Criar o gráfico
    plt.figure(figsize=(7, 3))
    ax = sns.barplot(x=summary["qtde."], y=summary.index, color="blue")  
    
    # Títulos e rótulos
    plt.title(f"Distribuição de '{col}'", color='white', fontsize=14)
    plt.xlabel("Quantidade", color='white', fontsize=12)
    plt.ylabel("", color='white', fontsize=12)
    
    # Ajustar cores e rótulos
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    
    # Adicionar os valores na frente das barras
    for container in ax.containers:
        ax.bar_label(container, fmt='%d', color='white', fontsize=10, padding=4)
    
    # Remover a grade
    ax.grid(False)
    
    plt.show()


In [ ]:
#verificar informações do df
production_product.info()

In [ ]:
# Variáveis quantitativas*: 

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['safetystocklevel', 'reorderpoint', 'standardcost', 'listprice', 'weight', 'daystomanufacture']

# Estatísticas Descritivas das colunas numéricas**
print("Estatísticas Descritivas:")
print(production_product[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)***
for col in numeric_columns:
    q1 = production_product[col].quantile(0.25)
    q3 = production_product[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers****
    outliers = production_product[(production_product[col] < lower_bound) | (production_product[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])



#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas, no nosso caso, vacationhours, sickleavehours, por exemplo
#doc**: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc***: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc****: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados


In [ ]:
# Lista de variáveis com outliers
variables_with_outliers = ['standardcost', 'listprice', 'weight', 'daystomanufacture']

# Configurar o layout da grade
fig, axes = plt.subplots(2, 2, figsize=(15, 10))  # 2x2 grid
axes = axes.flatten()  # Para facilitar o acesso aos eixos

# Criar os gráficos
for i, var in enumerate(variables_with_outliers):
    ax = sns.boxplot(
        y=production_product[var],
        color="blue",
        boxprops={"color": "blue"}, whiskerprops={"color": "blue"}, capprops={"color": "blue"},
        medianprops={"color": "red"}, flierprops={"markerfacecolor": "green", "markersize": 5},
        ax=axes[i]
    )
    
    ax.set_title(f'Distribuição de {var}', color='white', fontsize=14)
    ax.set_ylabel(var, color='white', fontsize=12)
    ax.set_xlabel("")
    
    # Ajustar as cores dos ticks
    ax.tick_params(axis='y', colors='white')
    ax.tick_params(axis='x', colors='white')
    
    # Remover a grade (opcional)
    ax.grid(False)

# Ajustar o layout para evitar sobreposição
plt.tight_layout()
plt.show()


#doc*****: de acordo com os resultados, as colunas safetystocklevel e reorderpoint não há necessidade de tratamento, sem outliers
#          standardcost: 55 outliers acima de 792.69, o que pode ser aceitos, pois podem efletir produtos premium ou com custos elevados de produção
#          listprice: Também identificados 55 outliers acima de 1412.47. Isso é esperado, já que produtos com custo alto podem ter preços elevados
#          daystomanufacture: detectados 97 outliers acima de 2.5 dias. Esses valores (ex.: 4 dias) podem ser válidos para produtos mais complexos ou com maior tempo de produção

In [ ]:
#Verificando os outliers de da coluna weight: valores muito elevados para os componentes

# Calcular os limites para a coluna weight
q1 = production_product['weight'].quantile(0.25)
q3 = production_product['weight'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filtrar valores que ultrapassam os limites
outliers_weight = production_product[
    (production_product['weight'] < lower_bound) | 
    (production_product['weight'] > upper_bound)
]

# Ordenar os outliers por weight em ordem decrescente
outliers_weight_sorted = outliers_weight[['productid', 'name', 'weight']].sort_values(by='weight', ascending=False)

# Exibir os valores
print(outliers_weight_sorted)


In [ ]:
# Categorias de produtos com outliers em 'weight'
categories_with_outliers = outliers_weight.groupby('name')['weight'].mean().sort_values(ascending=False)

# Exibir categorias com peso médio para revisão
print(categories_with_outliers)


#doc:uma lista de nomes de produtos (ex.: "LL Road Rear Wheel", "ML Crankset") e seus pesos médios; 
#    visão consolidada para validar quais categorias têm valores incoerentes

In [ ]:
# Definir valores aproximados por categoria (em kg)
weight_corrections = {
    "LL Road Rear Wheel": 1.5,
    "ML Road Rear Wheel": 1.5,
    "LL Road Front Wheel": 1.2,
    "HL Road Rear Wheel": 1.5,
    "ML Road Front Wheel": 1.2,
    "HL Road Front Wheel": 1.2,
    "ML Crankset": 1.0,
    "LL Crankset": 1.0,
    "HL Crankset": 1.0,
    "Touring Rim": 0.7,
    "ML Road Rim": 0.7,
    "ML Mountain Rim": 0.8,
    "LL Road Rim": 0.7,
    "LL Mountain Rim": 0.8,
    "HL Mountain Rim": 0.8,
    "HL Road Rim": 0.7,
    "Rear Brakes": 0.3,
    "Front Brakes": 0.3,
    "LL Bottom Bracket": 0.4,
    "LL Mountain Pedal": 0.4,
    "ML Mountain Pedal": 0.4,
    "Rear Derailleur": 0.5,
    "LL Road Pedal": 0.3,
    "HL Mountain Pedal": 0.4,
    "HL Bottom Bracket": 0.4,
    "ML Bottom Bracket": 0.4,
    "ML Road Pedal": 0.3,
    "HL Road Pedal": 0.3,
    "Front Derailleur": 0.3,
}


In [ ]:
# Aplicar correções no DataFrame
production_product['weight_corrected'] = production_product['name'].map(weight_corrections).fillna(production_product['weight'])

# Verificar os resultados
print(production_product[['name', 'weight', 'weight_corrected']].sort_values(by='weight', ascending=False).head(20))


# Correção de valores inconsistentes na coluna 'weight'

# 1. Identificação de inconsistências
# Durante a análise, algumas categorias de produtos apresentavam valores de peso incoerentes com a realidade, como rodas com 1050 kg e cranksets com 635 kg. 
# Esses valores foram considerados erros de entrada de dados.

# 2. Definição de pesos aproximados
# Foi criado um dicionário chamado 'weight_corrections' contendo valores aproximados para cada
# categoria de produto com base em características reais de produtos similares. Esses valores
# representam estimativas coerentes dentro do domínio de produtos da Adventure Works.

# 3. Aplicação das correções
# Os valores inconsistentes foram substituídos pelos valores definidos no dicionário.
# Para isso, a função 'map()' foi utilizada para associar os valores corrigidos às categorias
# específicas, criando uma nova coluna chamada 'weight_corrected'.
# Caso um produto não estivesse no dicionário de correções, o valor original foi mantido.

# 4. Resultado
# A nova coluna 'weight_corrected' foi criada, contendo os valores ajustados.
# Agora, essa coluna pode ser utilizada para análises e exportação, enquanto a coluna original
# 'weight' pode ser mantida para rastreamento e auditoria, se necessário.

In [ ]:
# Reorganizar as colunas no DataFrame para que weight_corrected fique ao lado de weight
cols = production_product.columns.tolist()
if 'weight' in cols and 'weight_corrected' in cols:
    weight_index = cols.index('weight')
    cols.insert(weight_index + 1, cols.pop(cols.index('weight_corrected')))
production_product = production_product[cols]

# Verificar as colunas reorganizadas
print(production_product.columns)


In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['production_product'] = production_product

In [ ]:
# Padronizar colunas com valores textuais
production_product['name'] = production_product['name'].str.strip().str.upper()
production_product['productnumber'] = production_product['productnumber'].str.strip().str.upper()
production_product['color'] = production_product['color'].str.strip().str.upper()
production_product['weightunitmeasurecode'] = production_product['weightunitmeasurecode'].str.strip().str.upper()
production_product['productline'] = production_product['productline'].str.strip().str.upper()
production_product['class'] = production_product['class'].str.strip().str.upper()
production_product['size'] = production_product['size'].str.strip().str.upper()
production_product['rowguid'] = production_product['rowguid'].str.strip().str.upper()

print(production_product.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")